# The Titanic data challenge

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Settings
showPlots = True

In [ ]:
# Load the data
train_df = pd.read_csv('data/train.csv')
apply_df = pd.read_csv('data/test.csv')
print(train_df[:3], '\n')
print(train_df.info())

## Test submissions
Test files:
* random prediction
* survival for women and children only

In [ ]:
createRandom, createWomenChildren = False, False

In [ ]:
if createRandom:
    # Create dummy random output for first submission
    id_df = apply_df['PassengerId']
    random_df = pd.DataFrame(np.random.randint(low=0, high=2, size=(id_df.shape[0], 1)), columns=['Survived'])
    result_df = pd.concat([id_df, random_df], axis=1)

    # Save output to file
    result_df.to_csv("results/result_random.csv", index=False, float_format='%.0f')

In [ ]:
if createWomenChildren:
    # Create dummy output with survival for women and children only
    result_df = apply_df
    result_df['Survived'] = ((result_df['Sex'] == 'female') | (result_df['Age'] < 16)).astype(int)

    # Save output to file
    result_df.to_csv("results/result_children_women.csv", columns=['PassengerId', 'Survived'], index=False, float_format='%.0f')

## Investigate the data

In [ ]:
# Have a general look
train_df.describe()

In [ ]:
# Find how many unique entries
train_df.describe(include='O')

In [ ]:
# Count how many men
train_df.loc[train_df['Sex'] == 'male'].shape[0]

## Data preparation

### Remove unusable data
* drop PassengerId, Ticket

In [ ]:
X_full = train_df.drop(["Survived", "PassengerId", "Ticket"], axis=1)
y_full = train_df["Survived"]
X_apply = apply_df.drop(["PassengerId", "Ticket"], axis=1)
X_full.head()

### Transform non-numeric labels
* fill NaN values with sensible defaults
* fill missing values with medians
* integer labels for 'Sex', 'Embarked' and 'Cabin'

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


class MyNumericizer(BaseEstimator, TransformerMixin):
    def __init__(self, add_cabin_info=True):
        # no *args, **kargs to make use of BaseEstimator class
        # other args can be steered later as hyperparameters
        self.add_cabin_info = add_cabin_info

    def fit(self, X, y=None):
        return self  # nothing to do

    def transform(self, X, y=None):
        # Sex
        binarizer = LabelBinarizer(sparse_output=False)
        col = X['Sex']
        col = pd.DataFrame(binarizer.fit_transform(col), columns=['Sex'])
        X = X.drop('Sex', axis=1)
        X = pd.concat([X, col], axis=1)
        # Embarked
        encoder = LabelEncoder()
        X['Embarked'].fillna('unknown', inplace=True)
        col = encoder.fit_transform(X['Embarked'])
        col = pd.DataFrame(col, columns=['Embarked'])
        X = X.drop('Embarked', axis=1)
        X = pd.concat([X, col], axis=1)
        #cabin
        if self.add_cabin_info:
            encoder = LabelEncoder()
            X['Cabin'].fillna('unknown', inplace=True)
            col = encoder.fit_transform(X['Cabin'])
            col = pd.DataFrame(col, columns=['Cabin'])
            X = X.drop('Cabin', axis=1)
            X = pd.concat([X, col], axis=1)
        else:
            X = X.drop(['Cabin'], axis=1)
        #Age
        median = X['Age'].median()
        X['Age'].fillna(median, inplace=True)
        #Fare
        median = X['Fare'].median()
        X['Fare'].fillna(median, inplace=True)
        
        if X.isnull().any().any():
            print('Warning: null value detected:')
            print(X.isnull().any())
            
        return X

# print(X_new.head())
# scaler = StandardScaler()
# X_new = scaler.fit_transform(X_new)
# print(pd.DataFrame(X_new, columns = X_full.columns).head())


### Visualization

#### Before transformation to numerical values

In [ ]:
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

%matplotlib inline

if showPlots:
    plt.figure()
    X_full.hist(bins=50, figsize=(20, 15))
    1  # prevent matlpotlib printout

#### After transformation to numerical values

In [ ]:
my_num = MyNumericizer()
X_plot = my_num.fit_transform(X_full)

if showPlots:
    plt.figure()
    X_plot.hist(bins=50, figsize=(20, 15))
    1  # prevent matlpotlib printout

#### Correlation plots

In [ ]:
if showPlots:
    plt.figure()
    scatter_matrix(pd.concat([X_plot, y_full], axis=1), figsize=(12, 8))
    1  # prevent matlpotlib printout

#### Take a closer look at single features

In [ ]:
# import seaborn as sns

# if showPlots:
#     g = sns.FacetGrid(train_df, col='Survived')
#     g.map(plt.hist, 'Age', bins=20)

In [ ]:
# if showPlots:
#     # grid = sns.FacetGrid(train_df, col='Pclass', hue='Survived')
#     grid = sns.FacetGrid(train_df, col='Survived', row='Pclass', size=2.2, aspect=1.6)
#     grid.map(plt.hist, 'Age', alpha=.5, bins=20)
#     grid.add_legend()

In [ ]:
# if showPlots:
#     # grid = sns.FacetGrid(train_df, col='Embarked')
#     grid = sns.FacetGrid(train_df, row='Embarked', size=2.2, aspect=1.6)
#     grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette='deep')
#     grid.add_legend()

In [ ]:
# if showPlots:
#     # grid = sns.FacetGrid(train_df, col='Embarked', hue='Survived', palette={0: 'k', 1: 'w'})
#     grid = sns.FacetGrid(train_df, row='Embarked', col='Survived', size=2.2, aspect=1.6)
#     grid.map(sns.barplot, 'Sex', 'Fare', alpha=.5, ci=None)
#     grid.add_legend()

### Feature engineering
* number of family members
* coarser binning for age?

In [ ]:
class MyFeatureAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_relatives=True, add_nobility=False):
        self.add_nobility = add_nobility
        self.add_relatives = add_relatives
    
    def fit(self, X, y=None):
        return self  # nothing to do

    def transform(self, X, y=None):
        if self.add_nobility:
            def extract_nobility(row):
                name = row['Name']
                if 'Lady' in name:
                    return 1
                if 'Sir' in name:
                    return 1
                return 0
            
            X['Nobility'] = X.apply(lambda row: extract_nobility(row), axis=1)
        if self.add_relatives:
            X['Family'] = X['SibSp'] + X['Parch']
        X = X.drop('Name', axis=1)
        return X

feat_adder = MyFeatureAdder()
X_new = feat_adder.fit_transform(X_full)
print(X_new.head())
# X_new.loc[X_new['Nobility'] == 1].count()
# print(X_new['Name'].where(X_new['Pclass'] == 1).dropna())


### Create pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

add_cabin_info = False  # adding this reduces accuracy
add_relatives = True  # adding this increases accuracy
add_nobility = False  # in current implementation reduces accuracy

clfs = {}

clfs['linear_svc'] = Pipeline((
        ('MyNumericizer', MyNumericizer(add_cabin_info=add_cabin_info)),
        ('MyFeatureAdder', MyFeatureAdder(add_relatives=add_relatives, add_nobility=add_nobility)),
        ('scaler', StandardScaler()),
        ('linear_svc', LinearSVC(C=1, loss='hinge')),
        ))

clfs['rbf_svc'] = Pipeline((
        ('MyNumericizer', MyNumericizer(add_cabin_info=add_cabin_info)),
        ('MyFeatureAdder', MyFeatureAdder(add_relatives=add_relatives, add_nobility=add_nobility)),
        ('scaler', StandardScaler()),
        ('rbf_svc', SVC(kernel='rbf', C=1)),
        ))

clfs['poly_svc'] = Pipeline((
        ('MyNumericizer', MyNumericizer(add_cabin_info=add_cabin_info)),
        ('MyFeatureAdder', MyFeatureAdder(add_relatives=add_relatives, add_nobility=add_nobility)),
        ('scaler', StandardScaler()),
        ('poly_svc', SVC(kernel='poly', C=1)),
        ))

clfs['tree'] = Pipeline((
        ('MyNumericizer', MyNumericizer(add_cabin_info=add_cabin_info)),
        ('MyFeatureAdder', MyFeatureAdder(add_relatives=add_relatives, add_nobility=add_nobility)),
        ('scaler', StandardScaler()),
        ('tree', DecisionTreeClassifier()),
        ))

clfs['kNN'] = Pipeline((
        ('MyNumericizer', MyNumericizer(add_cabin_info=add_cabin_info)),
        ('MyFeatureAdder', MyFeatureAdder(add_relatives=add_relatives, add_nobility=add_nobility)),
        ('scaler', StandardScaler()),
        ('kNN', KNeighborsClassifier()),
        ))

clfs['gradBoost'] = Pipeline((
        ('MyNumericizer', MyNumericizer(add_cabin_info=add_cabin_info)),
        ('MyFeatureAdder', MyFeatureAdder(add_relatives=add_relatives, add_nobility=add_nobility)),
        ('scaler', StandardScaler()),
        ('gradBoost', GradientBoostingClassifier(learning_rate=1.0, n_estimators=3, max_depth=2)),
        ))

clfs['randomForest'] = Pipeline((
        ('MyNumericizer', MyNumericizer(add_cabin_info=add_cabin_info)),
        ('MyFeatureAdder', MyFeatureAdder(add_relatives=add_relatives, add_nobility=add_nobility)),
        ('scaler', StandardScaler()),
        ('randomForest', RandomForestClassifier()),
        ))

## Training

### Split training sample into train and validation set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.25, random_state=1337)
# Reset the index to reach from 0 to n-1 to avoid NaN rows
for x in [X_train, X_test, y_train, y_test]:
    x.reset_index(drop=True, inplace=True)

### Training

In [ ]:
for name, clf in clfs.items():
    clf.fit(X_train, y_train)

## Evaluation

In [ ]:
from sklearn.metrics import accuracy_score

accuracies = []
for name, clf in clfs.items():
    y_pred = clf.predict(X_test)
    accuracy = round(accuracy_score(y_test, y_pred), 3)
    accuracies.append((name, accuracy))
    
accuracies.sort(key=lambda x: x[1], reverse=True)
for acc in accuracies:
    print(acc[0], ':', *acc[1:])

top = accuracies[0][0]

## Prediction

In [ ]:
y_pred = clfs[top].predict(X_apply)

In [ ]:
id_pred = pd.DataFrame(apply_df['PassengerId'])
y_pred = pd.DataFrame(y_pred, columns=['Survived'])
result_df = pd.concat([id_pred, y_pred], axis=1)
print(result_df.head())

# Save output to file
result_df.to_csv("results/result.csv", index=False, float_format='%.0f')